# Rheostat Biocrnpyler 4
## Implement Entire pathway

In [1]:
# import cell
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
from bokeh.layouts import column
#import bokeh_catplot
bokeh.io.output_notebook()

from biocrnpyler.mechanism import Mechanism
from biocrnpyler.component import Component
from biocrnpyler import Mixture
from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, ChemicalReactionNetwork
import numpy as np

import warnings
warnings.filterwarnings('ignore')

Loading BokehJS ...

In [2]:
def check_type(item, material_type_str): # call set_species
    if isinstance(item, Species):
        item_ret = item
    elif isinstance(item, str):
        item_ret = Species(name = item, material_type = material_type_str)
    elif isinstance(item, Component) and item.get_species() != None:
        item_ret = item.get_species()
    else:
        raise ValueError( item, "parameter must be a string, a Component with defined get_species(), or a chemical_reaction_network.species")
        
    return item_ret

In [3]:
class FuelMichaelisMenten(Mechanism):
    def __init__(self, name, type = 'catalysis', **keywords):
        
        Mechanism.__init__(self = self, name = name, mechanism_type = type, **keywords)

    def update_species(self, enzyme, fuel_list, substrate_list, product_list, waste_list): 
        
        self.enzyme = check_type(enzyme, 'enzyme')
        
        species = [self.enzyme]
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]
        
        for f in fuel_list:
            species.append(f)
            comp1_list.append(f)

        for s in substrate_list:
            species.append(s)
            comp1_list.append(s)
            
        for p in product_list:
            species.append(p)
            comp2_list.append(p)
            
        for w in waste_list:
            species.append(w)
            comp2_list.append(w)
               
        
        species += [ComplexSpecies(comp1_list)]
        species += [ComplexSpecies(comp2_list)]
        return species
    
    def update_reactions(self, enzyme, fuel_list, substrate_list, product_list, waste_list, k_bf, k_uf, k_cat, component, part_id = None):

       # k_bf = 22.68
        
        #k_bf = component.get_parameter('k_bf', part_id = part_id, mechanism = self)
        
        k_br = 0.01*k_bf
       # k_uf = 24
        k_ur = 0.01*k_uf
       # k_cat = 10
       # k_atp_use = 0.5
       # print(enzyme)
        self.enzyme = check_type(enzyme, 'enzyme')
        
        
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]


        for f in fuel_list:
            comp1_list.append(f)
            
        for s in substrate_list:
            comp1_list.append(s)

        for p in product_list:
            comp2_list.append(p)
            
        for w in waste_list:
            comp2_list.append(w)
        
        comp1 = ComplexSpecies(comp1_list)
        comp2 = ComplexSpecies(comp2_list)
        
        binding_rxn = Reaction(inputs = comp1_list, outputs=[comp1], k = k_bf, k_rev = k_br)
        cat_rxn = Reaction(inputs = [comp1], outputs = [comp2], k = k_cat)
        unbinding_rxn = Reaction(inputs = [comp2], outputs = comp2_list, k=k_uf, k_rev = k_ur)
        
        return [binding_rxn, cat_rxn, unbinding_rxn]

In [4]:
class Enzyme(Component):
    def __init__(self, enzyme_name, substrate, fuel, product, waste, k_bf, k_uf, k_cat = 3600, **keywords):
      
        # ENZYME NAME
        # self.enzyme = self.set_species()
        self.enzyme = check_type(enzyme_name, 'enzyme')
    
        # SUBSTRATE
        self.substrate_list = []
        for s in substrate:
            self.substrate_list.append(self.set_species(s, material_type = 'molecule'))
            
#         for s in substrate:
#             setattr(self, f'{s}', check_type(s, 'molecule'))
#             substrate_list.append(getattr(self, f'{s}'))
            
#         self.substrate_list = substrate_list
        
        
        # FUEL
        
        self.fuel_list = []
        for f in fuel:
            self.fuel_list.append(self.set_species(f, material_type ='metabolite'))
        
        
        
        # PRODUCT
        self.product_list = []
        for p in product:
            self.product_list.append(self.set_species(p, material_type = 'molecule'))
          
            
        # WASTE
        self.waste_list = []
        for w in waste:
            self.waste_list.append(self.set_species(w, material_type = 'metabolite'))
        
        
        self.k_bf = k_bf
        self.k_uf = k_uf
        self.k_cat = k_cat
        
            
        
      
        Component.__init__(self = self, name = enzyme_name, **keywords)
        
    def update_species(self):
        mech_cat = self.mechanisms['catalysis']
        
            
        
        return mech_cat.update_species(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list) 
                                                                                           
    
    def update_reactions(self):
        mech_cat = self.mechanisms['catalysis']

        
        return mech_cat.update_reactions(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list,self.k_bf,
                                         self.k_uf, self.k_cat,
                                             component = self, part_id = self.name) # for parameters



In [5]:
class EnergyTxTl(Mixture):
    def __init__(self, name="",**keywords): 
        

        mech_cat = FuelMichaelisMenten('catalysis')
        
        default_mechanisms = {
            mech_cat.mechanism_type:mech_cat
        }
        
        #species = [self.enzyme]
        Mixture.__init__(self, name = name, default_mechanisms=default_mechanisms, **keywords) 
        #  parameter_file = 'parameters.txt'
    

# Using parameters from the paper

In [6]:
# parameter dictionary (choose one, dont mix)
# parameters = {("catalysis","enzyme1", "k_bf"):1.0, 
#               ("default_gene_expression","ReporterBLAH", "ku"):.01,
#               ("default_gene_expression","ReporterBLAH", "kb"):100.0 }


E1_hex = Enzyme(enzyme_name = "hex", substrate = ['glucose'],
            fuel = ['atp'],product = ['g6p'], waste = ['adp'], k_bf = 0.08*3600 , k_uf = 0.2*3600)

E2_pgi = Enzyme(enzyme_name = 'pgi', substrate = ['g6p'], fuel = [],
           product = ['f6p'], waste = [], k_bf = 1.4*3600, k_uf = 0.3*3600)

E3_pfk = Enzyme(enzyme_name = 'pfk', substrate = ['f6p'], fuel = ['atp'], product = ['f16p'],
           waste = ['adp'], k_bf = 0.1*3600, k_uf = 0.1*3600)

E4_ald_tpi = Enzyme(enzyme_name ='ald_tpi' , substrate = ['f16p'], fuel = [], product = ['g3p', 'g3p'], 
            waste = [], k_bf = 0.3*3600 , k_uf = 0.3*3600 )

E5_gapN = Enzyme(enzyme_name ='gapN' , substrate = ['g3p', 'g3p'], fuel = ['nadp', 'nadp'], product = ['3pg', '3pg'], 
            waste = ['nadph', 'nadph'], k_bf = 0.1*3600, k_uf = 0.3*3600  )

E6_gapM6 = Enzyme(enzyme_name ='gapM6' , substrate = ['g3p', 'g3p'], fuel = ['pi'], product = ['13bpg'], 
            waste = [], k_bf =0.3*3600 , k_uf = 0.3*3600)

E7_pgk = Enzyme(enzyme_name = 'pgk', substrate = ['13bpg'], fuel = ['adp'], product = ['3pg', '3pg'], 
            waste = ['atp'], k_bf = 0.03*3600 , k_uf = 0.03*3600 )

E8_pgm = Enzyme(enzyme_name ='pgm' , substrate = ['3pg', '3pg'], fuel = [], product = ['2pg', '2pg'], 
            waste = [], k_bf = 0.2*3600, k_uf = 0.08 *3600)

E9_eno = Enzyme(enzyme_name ='eno' , substrate = ['2pg', '2pg'], fuel = [], product = ['pep', 'pep'], 
            waste = [], k_bf = 0.04*3600 , k_uf =  0.5*3600)

E10_pyk = Enzyme(enzyme_name = 'pyk', substrate = ['pep', 'pep'], fuel = ['adp', 'adp'], product = ['pyruvate', 'pyruvate'], 
            waste = ['atp', 'atp'], k_bf = 0.14*3600 , k_uf = 0.000001) # irreversible

E11_alsS = Enzyme(enzyme_name = 'alsS', substrate = ['pyruvate', 'pyruvate'], fuel = [], product = ['acetolac'], 
            waste = [], k_bf = 3.2*3600 , k_uf = 0.000001) # irreversible

E12_IlvC = Enzyme(enzyme_name = 'IlvC', substrate = ['acetolac'], fuel = ['nadph'], product = ['23dih3mebut'], 
            waste = ['nadp'], k_bf =0.3 *3600, k_uf = 0.3 *3600)

E13_IlvD = Enzyme(enzyme_name ='IlvD' , substrate = ['23dih3mebut'], fuel = [], product = ['3me2oxo'], 
            waste = [], k_bf = 0.3*3600, k_uf = 0.5 *3600)

E14_kivD = Enzyme(enzyme_name ='kivD' , substrate = ['3me2oxo'], fuel = [], product = ['isobutanal'], 
            waste = [], k_bf = 0.1*3600, k_uf = 0.000001*3600) # irreversible

E15_yahk = Enzyme(enzyme_name = 'yahk', substrate = ['isobutanal'], fuel = ['nadph'], product = ['isobutanol'],
                  waste = ['nadp'], k_bf = 0.1*3600, k_uf = 0.3*3600)


E16_all_other_atp = Enzyme(enzyme_name = 'atp_synthase', substrate = [], fuel = ['atp'], 
                           product = [], waste = ['adp', 'pi'], k_bf = 22.68*3600, k_uf = 24*3600, k_cat = 0.5*3600,)

myMixture = EnergyTxTl(components = [E1_hex,E2_pgi,E3_pfk, E4_ald_tpi, E5_gapN, E6_gapM6, E7_pgk, E8_pgm, E9_eno, E10_pyk, 
                                    E11_alsS, E12_IlvC, E13_IlvD, E14_kivD, E15_yahk, E16_all_other_atp])
CRN = myMixture.compile_crn()

myMixture_atp = EnergyTxTl(components = [E16_all_other_atp])
CRN_atp = myMixture_atp.compile_crn()
print(CRN.pretty_print(show_rates = False, show_attributes = False, show_materials = False))



Species (68) = {0. enzyme[hex], 1. metabolite[atp], 2. molecule[glucose], 3. molecule[g6p], 4. metabolite[adp], 5. complex[enzyme[hex]:metabolite[atp]:molecule[glucose]], 6. complex[enzyme[hex]:metabolite[adp]:molecule[g6p]], 7. enzyme[pgi], 8. molecule[f6p], 9. complex[enzyme[pgi]:molecule[g6p]], 10. complex[enzyme[pgi]:molecule[f6p]], 11. enzyme[pfk], 12. molecule[f16p], 13. complex[enzyme[pfk]:metabolite[atp]:molecule[f6p]], 14. complex[enzyme[pfk]:metabolite[adp]:molecule[f16p]], 15. enzyme[ald_tpi], 16. molecule[g3p], 17. complex[enzyme[ald_tpi]:molecule[f16p]], 18. complex[enzyme[ald_tpi]:2x_molecule[g3p]], 19. enzyme[gapN], 20. metabolite[nadp], 21. molecule[3pg], 22. metabolite[nadph], 23. complex[enzyme[gapN]:2x_metabolite[nadp]:2x_molecule[g3p]], 24. complex[enzyme[gapN]:2x_metabolite[nadph]:2x_molecule[3pg]], 25. enzyme[gapM6], 26. metabolite[pi], 27. molecule[13bpg], 28. complex[enzyme[gapM6]:metabolite[pi]:2x_molecule[g3p]], 29. complex[enzyme[gapM6]:molecule[13bpg]], 30. 

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/component.py:11: UserWarning: get_species() not defined for component {self.name}, None returned.
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/component.py:11: UserWarning: get_species() not defined for component {self.name}, None returned.


In [168]:
CRN.write_sbml_file("CRN.sbml")
CRN_atp.write_sbml_file("CRN_atp.sbml")
timepoints = np.linspace(0,72,100)
e = 0.15
e4 = 3.4
atp = 30
x0 = {'molecule_glucose':30,
      'metabolite_atp': atp,
     'metabolite_nadp':atp,
      "enzyme_hex":e,
      'enzyme_pgi':e,
      'enzyme_pfk':e,
      'enzyme_ald_tpi':e,
      'enzyme_gapN':e, 
      'enzyme_gapM6':e,
      'enzyme_pgk':e,
      'enzyme_pgm':e,
      'enzyme_eno':e,
      'enzyme_pyk':e,
      'enzyme_alsS':e,
      'enzyme_IlvC':e,
      'enzyme_IlvD':e,
      'enzyme_kivD':e,
      'enzyme_yahk':e,
      "enzyme_atp_synthase":e4
}
     
x0_atp = {'metabolite_atp':atp,
          "enzyme_atp_synthase":e4,
}

re = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)
re_atp = CRN_atp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atp)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...odeint failed with mxstep=500...

In [170]:
# same color scheme as the others for presentation
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
#colors = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']

# First plot glucose and isobutanol
p1 = bokeh.plotting.figure(width = 450, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration (mM)',
                         title = 'Entire Pathway Biocrnpyler with paper parameters')
p1.line(timepoints, re['molecule_glucose'], color = colors[0],line_width = 2, legend_label = 'glucose')
p1.line(timepoints, re['molecule_isobutanol'], color = colors[1], line_width = 2,legend_label = 'isobutanol')
p1.line(timepoints, re['molecule_f16p'], color = colors[2], line_width = 2,legend_label = 'f16p')
p1.legend.location = 'center_right'
p1.legend.click_policy="hide"


# Plot ATP and ADP, Pi
p2 = bokeh.plotting.figure(width = 450, height = 250,
                           x_axis_label = 'time (hrs)',
                         y_axis_label = 'concentration')
p2.line(timepoints, re['metabolite_atp'], color = colors[3], line_width = 2,legend_label = 'atp')
p2.line(timepoints, re['metabolite_pi'], color = colors[6],line_width = 2, legend_label = 'pi')
p2.line(timepoints, re['metabolite_adp'], color = colors[5], line_width = 2,legend_label = 'adp')
p2.legend.location = 'center_right'
p2.legend.click_policy="hide"


p2.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp only',line_width=2, color = colors[4])
bokeh.io.show(row(p1,p2))




In [151]:
#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f']
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02', '#a6761d']
p1 = bokeh.plotting.figure(width =450, height = 250, title = 'Dynamics rxn 1,2,3', x_axis_label = 'time',
                          y_axis_label = 'concentration')
p1.line(timepoints, re['metabolite_atp'], legend_label = 'atp', line_width = 2, color = colors[0])
p1.line(timepoints, re['metabolite_nadp'], legend_label = 'nadp', line_width = 2, color = colors[0])


p1.line(timepoints, re['molecule_glucose'], legend_label = 'glucose', line_width = 2, color = colors[1])
p1.line(timepoints, re['enzyme_hex'], legend_label = 'enzyme_hex', line_width = 2, color = colors[2])
p1.line(timepoints, re['metabolite_adp'], legend_label = 'adp', line_width = 2, color = colors[3])
p1.line(timepoints, re['molecule_f16p'], legend_label = 'f16p', line_width = 2, color = colors[4])
p1.line(timepoints, re['molecule_isobutanol'], legend_label = 'isobutanol', line_width = 2, color = colors[5])
p1.line(timepoints, re['metabolite_pi'], legend_label = 'pi', line_width = 2, color = colors[6])

p1.legend.click_policy="hide"

p2 = bokeh.plotting.figure(width =450, height = 250, title = 'Dynamics rxn atp leak only', x_axis_label = 'time',
                          y_axis_label = 'concentration', y_range = p1.y_range)
p2.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp', line_width = 2, color = colors[0])
p2.line(timepoints, re_atp['metabolite_adp'], legend_label = 'adp', line_width = 2, color = colors[3])
p2.line(timepoints, re_atp['enzyme_atp_synthase'], legend_label = 'enzyme_atp',line_width = 2, color = colors[2])
p2.legend.click_policy="hide"

p3 = bokeh.plotting.figure(width =450, height = 250, title = 'Dynamics rxn atp leak for both', x_axis_label = 'time',
                          y_axis_label = 'concentration', y_range = p1.y_range)

p3.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp_ONLY', line_width = 2, color = colors[0])
p3.line(timepoints, re['metabolite_atp'], legend_label = 'atp_rheostat', line_width = 2, color = colors[1])


p4 = bokeh.plotting.figure(width =450, height = 250, title = 
                           'Only enzyme', x_axis_label = 'time',
                          y_axis_label = 'concentration')
p4.line(timepoints, re['enzyme_hex'], legend_label = 'enzyme_hex', line_width = 2, color = colors[2])


bokeh.io.show(row(p1,p2))
bokeh.io.show(row(p3, p4))

Why isn't isobutanol growing

In [152]:
re

,enzyme_hex,metabolite_atp,molecule_glucose,molecule_g6p,metabolite_adp,complex_enzyme_hex_metabolite_atp_molecule_glucose,complex_enzyme_hex_metabolite_adp_molecule_g6p,enzyme_pgi,molecule_f6p,complex_enzyme_pgi_molecule_g6p,...,complex_enzyme_kivD_molecule_3me2oxo,complex_enzyme_kivD_molecule_isobutanal,enzyme_yahk,molecule_isobutanol,complex_enzyme_yahk_metabolite_nadph_molecule_isobutanal,complex_enzyme_yahk_metabolite_nadp_molecule_isobutanol,enzyme_atp_synthase,complex_enzyme_atp_synthase_metabolite_atp,complex_enzyme_atp_synthase_metabolite_adp_metabolite_pi,time
0,0.150000,30.000000,30.000000,0.000000,0.000000,0.000000e+00,0.000000,0.150000,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.15,0.000000e+00,0.000000e+00,0.000000e+00,3.400000,0.000000,0.000000,0.000000
1,0.000494,25.311789,29.829220,0.019768,1.154662,8.296141e-06,0.149497,0.148596,0.000096,1.141876e-06,...,1.117779e-70,1.093535e-68,0.15,4.700713e-95,6.970404e-95,6.286019e-93,0.015963,3.313160,0.070876,0.727273
2,0.000520,24.093852,29.807502,0.037222,2.373295,8.293667e-06,0.149472,0.144976,0.000676,2.098160e-06,...,4.009444e-61,8.973825e-59,0.15,2.007458e-81,5.968160e-82,1.195969e-79,0.016719,3.305702,0.077579,1.454545
3,0.000547,22.884963,29.785792,0.052434,3.582705,8.290825e-06,0.149444,0.140062,0.001967,2.855655e-06,...,6.559129e-56,2.121833e-53,0.15,3.885845e-74,5.546743e-75,1.606418e-72,0.017517,3.292677,0.089806,2.181818
4,0.000578,21.687695,29.764089,0.065884,4.780188,8.287584e-06,0.149414,0.134534,0.004016,3.446654e-06,...,2.764766e-52,1.153278e-49,0.15,6.198477e-69,5.528597e-70,2.015982e-67,0.018359,3.273491,0.108150,2.909091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.135131,0.001874,28.786650,0.019605,26.580061,1.707581e-07,0.014869,0.122903,0.995533,9.285566e-07,...,1.155232e-21,2.074914e-17,0.15,1.399824e-25,2.328435e-29,1.865320e-25,0.049949,0.000765,3.349285,69.090909
96,0.135824,0.001881,28.786224,0.018343,26.580551,1.551138e-07,0.014176,0.123668,0.998659,8.901809e-07,...,1.314747e-21,2.397861e-17,0.15,1.694887e-25,2.763510e-29,2.235405e-25,0.049949,0.000768,3.349283,69.818182
97,0.136474,0.001887,28.785795,0.017181,26.581000,1.650576e-07,0.013526,0.124385,1.001600,8.307726e-07,...,1.493245e-21,2.765017e-17,0.15,2.046131e-25,3.265057e-29,2.671494e-25,0.049949,0.000771,3.349280,70.545455
98,0.137083,0.001893,28.785362,0.016111,26.581410,1.660825e-07,0.012916,0.125057,1.004366,7.834610e-07,...,1.692614e-21,3.181596e-17,0.15,2.463114e-25,3.849979e-29,3.184040e-25,0.049949,0.000774,3.349277,71.272727


In [153]:
#print(re)

In [154]:
# plot all complexes
#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f']
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02', '#a6761d']
p1 = bokeh.plotting.figure(width =1000, height = 1000, title = 'Dynamics rxn 1,2,3', x_axis_label = 'time',
                          y_axis_label = 'concentration')
p1.line(timepoints, re['complex_enzyme_hex_metabolite_atp_molecule_glucose'], 
        legend_label = 'complex_enzyme_hex_metabolite_atp_molecule_glucose', line_width = 2, color = colors[0])

p1.line(timepoints, re['complex_enzyme_hex_metabolite_adp_molecule_g6p'], 
        legend_label = 'complex_enzyme_hex_metabolite_adp_molecule_g6p', line_width = 2, color = colors[1])

p1.line(timepoints, re['complex_enzyme_pgi_molecule_g6p'], 
        legend_label = 'complex_enzyme_pgi_molecule_g6p', line_width = 2, color = colors[2])

p1.line(timepoints, re['complex_enzyme_kivD_molecule_3me2oxo'], 
        legend_label = 'complex_enzyme_kivD_molecule_3me2oxo', line_width = 2, color = colors[3])

p1.line(timepoints, re['complex_enzyme_kivD_molecule_isobutanal'],
        legend_label = 'complex_enzyme_kivD_molecule_isobutanal', line_width = 2, color = colors[4])

p1.line(timepoints, re['complex_enzyme_yahk_metabolite_nadph_molecule_isobutanal'], 
        legend_label = 'complex_enzyme_yahk_metabolite_nadph_molecule_isobutanal', line_width = 2, color = colors[5])

p1.line(timepoints, re['complex_enzyme_yahk_metabolite_nadp_molecule_isobutanol'],
        legend_label = 'complex_enzyme_yahk_metabolite_nadp_molecule_isobutanol', line_width = 2, color = colors[6])

p1.line(timepoints, re['complex_enzyme_atp_synthase_metabolite_atp'], 
        legend_label = 'complex_enzyme_atp_synthase_metabolite_atp', line_width = 2, color = colors[0])


p1.line(timepoints, re['complex_enzyme_atp_synthase_metabolite_adp_metabolite_pi'], 
        legend_label = 'complex_enzyme_atp_synthase_metabolite_adp_metabolite_pi', line_width = 2, color = colors[1])


p1.legend.click_policy="hide"
bokeh.io.show(p1)

# Using previously proposed parameters

In [156]:
# parameter dictionary (choose one, dont mix)
# parameters = {("catalysis","enzyme1", "k_bf"):1.0, 
#               ("default_gene_expression","ReporterBLAH", "ku"):.01,
#               ("default_gene_expression","ReporterBLAH", "kb"):100.0 }
k_bf = 918000 #660
k_uf = 111600 #144 # per hour

#

E1_hex = Enzyme(enzyme_name = "hex", substrate = ['glucose'],
            fuel = ['atp'],product = ['g6p'], waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

E2_pgi = Enzyme(enzyme_name = 'pgi', substrate = ['g6p'], fuel = [],
           product = ['f6p'], waste = [], k_bf = k_bf , k_uf = k_uf)

E3_pfk = Enzyme(enzyme_name = 'pfk', substrate = ['f6p'], fuel = ['atp'], product = ['f16p'],
           waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

E4_ald_tpi = Enzyme(enzyme_name ='ald_tpi' , substrate = ['f16p'], fuel = [], product = ['g3p', 'g3p'], 
            waste = [], k_bf = k_bf , k_uf = k_uf )

E5_gapN = Enzyme(enzyme_name ='gapN' , substrate = ['g3p', 'g3p'], fuel = ['nadp', 'nadp'], product = ['3pg', '3pg'], 
            waste = ['nadph', 'nadph'], k_bf = k_bf , k_uf = k_uf)

E6_gapM6 = Enzyme(enzyme_name ='gapM6' , substrate = ['g3p', 'g3p'], fuel = ['pi'], product = ['13bpg'], 
            waste = [],k_bf = k_bf , k_uf = k_uf)

E7_pgk = Enzyme(enzyme_name = 'pgk', substrate = ['13bpg'], fuel = ['adp'], product = ['3pg', '3pg'], 
            waste = ['atp'], k_bf = k_bf , k_uf = k_uf)

E8_pgm = Enzyme(enzyme_name ='pgm' , substrate = ['3pg', '3pg'], fuel = [], product = ['2pg', '2pg'], 
            waste = [], k_bf = k_bf , k_uf = k_uf)

E9_eno = Enzyme(enzyme_name ='eno' , substrate = ['2pg', '2pg'], fuel = [], product = ['pep', 'pep'], 
            waste = [],k_bf = k_bf , k_uf = k_uf)

E10_pyk = Enzyme(enzyme_name = 'pyk', substrate = ['pep', 'pep'], fuel = ['adp', 'adp'], product = ['pyruvate', 'pyruvate'], 
            waste = ['atp', 'atp'], k_bf = k_bf , k_uf = k_uf) # irreversible

E11_alsS = Enzyme(enzyme_name = 'alsS', substrate = ['pyruvate', 'pyruvate'], fuel = [], product = ['acetolac'], 
            waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

E12_IlvC = Enzyme(enzyme_name = 'IlvC', substrate = ['acetolac'], fuel = ['nadph'], product = ['23dih3mebut'], 
            waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)

E13_IlvD = Enzyme(enzyme_name ='IlvD' , substrate = ['23dih3mebut'], fuel = [], product = ['3me2oxo'], 
            waste = [],k_bf = k_bf , k_uf = k_uf)

E14_kivD = Enzyme(enzyme_name ='kivD' , substrate = ['3me2oxo'], fuel = [], product = ['isobutanal'], 
            waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

E15_yahk = Enzyme(enzyme_name = 'yahk', substrate = ['isobutanal'], fuel = ['nadph'], product = ['isobutanol'],
                  waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)


E16_all_other_atp = Enzyme(enzyme_name = 'atp_synthase', substrate = [], fuel = ['atp'], 
                           product = [], waste = ['adp', 'pi'], k_bf = k_bf, k_uf = k_uf, k_cat = 1800,)

myMixture = EnergyTxTl(components = [E1_hex,E2_pgi,E3_pfk, E4_ald_tpi, E5_gapN, E6_gapM6, E7_pgk, E8_pgm, E9_eno, E10_pyk, 
                                    E11_alsS, E12_IlvC, E13_IlvD, E14_kivD, E15_yahk, E16_all_other_atp])
CRN = myMixture.compile_crn()

myMixture_atp = EnergyTxTl(components = [E16_all_other_atp])
CRN_atp = myMixture_atp.compile_crn()
#print(CRN.pretty_print(show_rates = False, show_attributes = False, show_materials = False))

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/component.py:11: UserWarning: get_species() not defined for component {self.name}, None returned.
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/component.py:11: UserWarning: get_species() not defined for component {self.name}, None returned.


In [159]:
CRN.write_sbml_file("CRN.sbml")
CRN_atp.write_sbml_file("CRN_atp.sbml")
timepoints = np.linspace(0,72,200)
e = 5 #0.2 # 0.15 # conc of enzymes in tx tl
e4 = 5 #65 #3.4
atp = 1.5
x0 = {'molecule_glucose':30,
      'metabolite_atp': atp,
     'metabolite_nadp':atp,
      "enzyme_hex":e,
      'enzyme_pgi':e,
      'enzyme_pfk':e,
      'enzyme_ald_tpi':e,
      'enzyme_gapN':e, 
      'enzyme_gapM6':e,
      'enzyme_pgk':e,
      'enzyme_pgm':e,
      'enzyme_eno':e,
      'enzyme_pyk':e,
      'enzyme_alsS':e,
      'enzyme_IlvC':e,
      'enzyme_IlvD':e,
      'enzyme_kivD':e,
      'enzyme_yahk':e,
      "enzyme_atp_synthase":e4
}
     
x0_atp = {'metabolite_atp':atp,
          "enzyme_atp_synthase":e4,
}

re = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)
re_atp = CRN_atp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atp)

odeint failed with mxstep=500...odeint failed with mxstep=500...

In [160]:
# same color scheme as the others for presentation
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
#colors = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']

# First plot glucose and isobutanol
p1 = bokeh.plotting.figure(width = 450, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration (mM)',
                         title = 'Entire Pathway Biocrnpyler with previously proposed parameters')
p1.line(timepoints, re['molecule_glucose'], color = colors[0],line_width = 2, legend_label = 'glucose')
p1.line(timepoints, re['molecule_isobutanol'], color = colors[1], line_width = 2,legend_label = 'isobutanol')
p1.line(timepoints, re['molecule_f16p'], color = colors[2], line_width = 2,legend_label = 'f16p')
p1.legend.location = 'center_right'

# Plot ATP and ADP, Pi
p2 = bokeh.plotting.figure(width = 450, height = 250,
                           x_axis_label = 'time (hrs)',
                         y_axis_label = 'concentration')
p2.line(timepoints, re['metabolite_atp'], color = colors[3], line_width = 2,legend_label = 'atp')
p2.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp leak',line_width=2, color = colors[4])


p2.line(timepoints, re['metabolite_pi'], color = colors[6],line_width = 2, legend_label = 'pi')
p2.line(timepoints, re['metabolite_adp'], color = colors[5], line_width = 2,legend_label = 'adp')
p2.legend.location = 'center_right'

bokeh.io.show(row(p1,p2))



In [43]:
#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f']
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02', '#a6761d']
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']


p1 = bokeh.plotting.figure(width =450, height = 350, title = 'Dynamics rxn 1,2,3', x_axis_label = 'time',
                          y_axis_label = 'concentration')
p1.line(timepoints, re['metabolite_atp'], legend_label = 'atp', line_width = 2, color = colors[3])
p1.line(timepoints, re['metabolite_nadp'], legend_label = 'nadp', line_width = 2, color = colors[6])
p1.line(timepoints, re['molecule_glucose'], legend_label = 'glucose', line_width = 2, color = colors[0])
p1.line(timepoints, re['enzyme_hex'], legend_label = 'enzyme_hex', line_width = 2, color = colors[6])
p1.line(timepoints, re['metabolite_adp'], legend_label = 'adp', line_width = 2, color = colors[6])
p1.line(timepoints, re['molecule_f16p'], legend_label = 'f16p', line_width = 2, color = colors[2])
p1.line(timepoints, re['molecule_isobutanol'], legend_label = 'isobutanol', line_width = 2, color = colors[1])
p1.line(timepoints, re['metabolite_pi'], legend_label = 'pi', line_width = 2, color = colors[6])

p1.legend.click_policy="hide"

p2 = bokeh.plotting.figure(width =450, height = 250, title = 'Dynamics rxn atp leak only', x_axis_label = 'time',
                          y_axis_label = 'concentration', y_range = p1.y_range)
p2.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp', line_width = 2, color = colors[3])
p2.line(timepoints, re_atp['metabolite_adp'], legend_label = 'adp', line_width = 2, color = colors[3])
p2.line(timepoints, re_atp['enzyme_atp_synthase'], legend_label = 'enzyme_atp',line_width = 2, color = colors[2])
p2.legend.click_policy="hide"

p3 = bokeh.plotting.figure(width =450, height = 250, title = 'Dynamics rxn atp leak for both', x_axis_label = 'time',
                          y_axis_label = 'concentration', y_range = p1.y_range)

p3.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp_ONLY', line_width = 2, color = colors[0])
p3.line(timepoints, re['metabolite_atp'], legend_label = 'atp_rheostat', line_width = 2, color = colors[1])


p4 = bokeh.plotting.figure(width =450, height = 250, title = 
                           'Only enzyme', x_axis_label = 'time',
                          y_axis_label = 'concentration')
p4.line(timepoints, re['enzyme_hex'], legend_label = 'enzyme_hex', line_width = 2, color = colors[2])


bokeh.io.show(row(p1,p2))
bokeh.io.show(row(p3, p4))